In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1588283181625_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#input path to business json file
input_path_business = 's3://jsonfileyelp/yelp_academic_dataset_business.json'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#reading business json file using spark
business_data_df = spark.read.json(input_path_business)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#converting into business dataframe
business_data_df.createOrReplaceTempView("business")
business_df = spark.sql("SELECT * FROM business")
business_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|           city|               hours|is_open|     latitude|      longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+---------------+--------------------+-------+-------------+---------------+--------------------+-----------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...|      Cornelius|[11:0-20:0, 10:0-...|      1|   35.4627242|    -80.8526119|The Range At Lake...|      28031|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|     Scottsdale|                null|  

In [5]:
#dropping all the unwanted columns
business_df = business_df.drop('hours','latitude','longitude','postal_code')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#choosing open only businesses
opened_business_data = business_df[business_df['is_open']==1]
opened_business_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+------------+-------+--------------------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|        city|is_open|                name|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+------------+-------+--------------------+------------+-----+-----+
|     10913 Bailey Rd|[,,,,,,, True,, T...|f9NumwFMBDn751xgF...|Active Life, Gun/...|   Cornelius|      1|The Range At Lake...|          36|  3.5|   NC|
|8880 E Via Linda,...|[,,,,,,,,,,, True...|Yzvjg0SayhoZgCljU...|Health & Medical,...|  Scottsdale|      1|   Carlos Santo, NMD|           4|  5.0|   AZ|
|3554 Rue Notre-Da...|                null|XNoUzKckATkOD1hP6...|Pets, Pet Service...|    Montreal|      1|             Felinus|           5|  5.0|   QC|
|  4827 E Downing Cir|[,,,,,,,,, True,,...|51M2Kk903DFYI6gnB...|Home Services, Pl.

In [14]:
#choosing business that belonged to automotive
automotive_df = opened_business_data.filter("categories like '%Automotive%'")
automotive_df = automotive_df.drop('is_open')
automotive_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+-----+-----+
|             address|          attributes|         business_id|          categories|      city|                name|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+----------+--------------------+------------+-----+-----+
|1720 W Elliot Rd,...|[,,,,,,,,, True,,...|cKyLV5oWZJ2NudWgq...|Auto Repair, Auto...|   Gilbert|Oasis Auto Center...|          38|  4.5|   AZ|
| 6910 E Southern Ave|[,,,,,,,,, True,,...|ScYkbYNkDgCneBrD9...|Auto Repair, Oil ...|      Mesa|Junction Tire & A...|          18|  5.0|   AZ|
|1110 Saw Mill Run...|[,,,,,,,,, True,,...|7Dv4_HAxsxvadEsT5...|Automotive, Auto ...|Pittsburgh|Dependable Brakes...|           5|  5.0|   PA|
|      2915 Cherry Rd|[,,,,,,,,, True,,...|edQoeeBFUTYGwnUSE...|Auto Customizatio...| Rock Hill|            Car Guyz|          10|  4.5|   SC|

In [15]:
print((automotive_df.count(), len(automotive_df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(13149, 9)

In [9]:
#reading the reviews json file
reviews_data_df = spark.read.json('s3://jsonfileyelp/yelp_academic_dataset_review.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
reviews_data_df.createOrReplaceTempView("reviews")

reviews_df = spark.sql("SELECT * FROM reviews")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
#converting into reviews dataframe
data_reviews = reviews_df.withColumnRenamed('stars','review_stars')

data_reviews.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|review_stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|-MhfebM0QIsKt87iD...|   0|2015-04-15 05:21:16|    0|xQY8N_XvtGbearJ5X...|         2.0|As someone who ha...|     5|OwjRMXRC0KyPrIlcj...|
|lbrU8StCq3yDfr-QM...|   0|2013-12-07 03:16:52|    1|UmFMZ8PyXZTY2Qcwz...|         1.0|I am actually hor...|     1|nIJD_7ZXHq-FX8byP...|
|HQl28KMwrEKHqhFrr...|   0|2015-12-05 03:18:11|    0|LG2ZaYiOgpr2DK_90...|         5.0|I love Deagan's. ...|     1|V34qejxNsCbcgD8C0...|
|5JxlZaqCnk1MnbgRi...|   0|2011-05-27 05:30:52|    0|i6g_oA9Yf9Y31qt0w...|         1.0|Dismal, lukewarm,...|     0|ofKDkJKXSKZXu5xJN...|
|IS4cv902ykd8wj1TR...|   0|2017-01-14 21:

In [19]:
#merging both business and reviews dataframe so as to get reviews of the automotive businesses.
joined_data = automotive_df.join(data_reviews, on=['business_id'], how='inner')

joined_data.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------------+--------------------+--------------------+-----------+--------------------+------------+-----+-----+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|         business_id|           address|          attributes|          categories|       city|                name|review_count|stars|state|cool|               date|funny|           review_id|review_stars|                text|useful|             user_id|
+--------------------+------------------+--------------------+--------------------+-----------+--------------------+------------+-----+-----+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+
|-Gh9a15ijNii-8rnp...|797 Bancroft Drive|                null|Auto Repair, Auto...|Mississauga|Volvo of Mississauga|           3|  3.5|   ON|   0|2018-04-28 22:15:36|    3|1UfmHhKEW0-FmuWdc...|         1.0|Had a horrible ex...|     

In [20]:
#writing into csv file

def toCSV(df,outputPath):
    df.coalesce(1).write.csv(outputPath)

joined_data = joined_data.drop('attributes','address','state','city')

toCSV(joined_data, "s3://jsonfileyelp/reviews1.csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…